In [4]:
import csv
import numpy as np
from numba import njit

def read_ecg(filename):
    with open(filename, mode="r") as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        result = []
        for row in csv_reader:
            result.append([float(row[0]), float(row[1])])
    return np.array(result)

In [5]:
data = read_ecg("AVA02_2017-01-18.csv")
data

array([[ 5.00000000e-03, -6.96634193e+03],
       [ 1.00000000e-02, -6.86817456e+03],
       [ 1.50000000e-02, -6.83211308e+03],
       ...,
       [ 3.14459900e+04, -7.67655276e+03],
       [ 3.14459950e+04, -7.67655276e+03],
       [ 3.14460000e+04, -7.67655276e+03]])